In [1]:
from dataset import train_dataloaded,valid_dataloaded
from main_model import data2vec_base
model_teacher=data2vec_base(is_teacher=True,masking_ratio=0)
model_student=data2vec_base(is_teacher=False)


c:\Users\bahaa\anaconda3\envs\altantorch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\bahaa\.cache\kagglehub\datasets\iamsouravbanerjee\animal-image-dataset-90-different-animals\versions\5


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR,CosineAnnealingWarmRestarts
from transformers import get_cosine_schedule_with_warmup
from tqdm import tqdm
from torch.optim import Adam

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
device

'cuda'

In [16]:
def train_model(model_teacher,model_student,train_dataloaded,valid_dataloaded):
    model_teacher=model_teacher.to("cuda")
    model_student=model_student.to("cuda")
    train_dataloaded=train_dataloaded
    valid_dataloaded=valid_dataloaded
    

    # Modelleri cihaza taşıyoruz
    num_epochs=30
    train_sets = len(train_dataloaded)
    warmup_epochs=5

    context_opt = torch.optim.AdamW(model_teacher.parameters(), lr=1e-4)
    predictor_opt = torch.optim.AdamW(model_student.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(predictor_opt,train_sets* warmup_epochs, train_sets * num_epochs)
    criterion = torch.nn.MSELoss()


    train_losses = []
    model_teacher.train()
    model_student.train()
    
    for epoch in range(num_epochs):
        total_loss = 0.0
        total_loss_val=0.0
        
        for idx, (x, _) in enumerate(tqdm(train_dataloaded, desc=f"Epoch {epoch+1}/{num_epochs}")):
            x = x.to(device)
            
            # context_model'den context, target ve target_indices elde ediliyor
            context= model_teacher(x)
            
            # target_model'den hedef temsiller (target representations) alınıyor
            context_student = model_student(x)
           
            
            # Her bir hedef temsil için loss hesaplanıyor
            
                # predictor_model ile context temsilleri elde ediliyor.
                # context.detach() ile geriye yayılımın target_model'e gitmesi engelleniyor.
                    

                        # Seçili patch'ler için uygun boyutta gürültü üret
        
                    
                    
                    # Loss: 1 - cosine_similarity, böylece negatif değer oluşması engelleniyor.
            loss = criterion(context, context_student)
                   
            
            
            
            
            
            
           
            # Geriye yayılım
            loss.backward()
            
            # Loss toplamına ekleme (skaler olarak)
            total_loss += loss.item()
            
            # Parametre güncellemesi
            context_opt.step()
            predictor_opt.step()
            
            context_opt.zero_grad()
            predictor_opt.zero_grad()
            if device == "cuda":
                torch.cuda.synchronize()
            
            # CUDA kullanıyorsak senkronizasyon
            
        for idx,(images,label) in enumerate(tqdm(valid_dataloaded, desc=f"Epoch {epoch+1}/{num_epochs}")):
            x = images.to(device)
            
            # context_model'den context, target ve target_indices elde ediliyor
            context = model_teacher(x)
            
            # target_model'den hedef temsiller (target representations) alınıyor
            context_student = model_student(x)
            
            with torch.no_grad():
            # Her bir hedef temsil için loss hesaplanıyor
                
                    # predictor_model ile context temsilleri elde ediliyor.
                    # context.detach() ile geriye yayılımın target_model'e gitmesi engelleniyor.
                    

                        # Seçili patch'ler için uygun boyutta gürültü üret
        
                    
                    
                    # Loss: 1 - cosine_similarity, böylece negatif değer oluşması engelleniyor.
                loss_i = criterion(context, context_student)
                   
                    
                    
                
                total_loss_val += loss_i.item()




                
                if device == "cuda":
                    torch.cuda.synchronize()
            # Öğrenme oranı güncellemesi
        scheduler.step()
        
        avg_loss = total_loss / len(train_dataloaded)
        avg_loss_v=total_loss_val/len(valid_dataloaded)
        train_losses.append(avg_loss)
        print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.6f} | Val Loss: {avg_loss_v:.6f}")
    
    print("Finished Training")
    return train_losses


In [17]:
model=train_model(model_teacher,model_student,train_dataloaded,valid_dataloaded)

Epoch 1/30: 100%|██████████| 34/34 [00:08<00:00,  4.05it/s]


Epoch 1 | Train Loss: 1.183993 | Val Loss: 1.182499


Epoch 2/30: 100%|██████████| 34/34 [00:08<00:00,  3.96it/s]


Epoch 2 | Train Loss: 1.181896 | Val Loss: 1.178311


Epoch 3/30: 100%|██████████| 34/34 [00:08<00:00,  4.01it/s]


...

Epoch 28 | Train Loss: 0.964499 | Val Loss: 0.962612


Epoch 29/30: 100%|██████████| 34/34 [00:08<00:00,  3.97it/s]


Epoch 29 | Train Loss: 0.962380 | Val Loss: 0.960411


Epoch 30/30: 100%|██████████| 34/34 [00:08<00:00,  3.92it/s]

Epoch 30 | Train Loss: 0.960256 | Val Loss: 0.958510
Finished Training


In [18]:
from reverse import Vec2Data
vec2data=Vec2Data()

In [ ]:
def train_model(model_teacher,model_student,vec2data,train_dataloaded,valid_dataloaded):
    model_teacher=model_teacher.to("cuda")
    model_student=model_student.to("cuda")
    train_dataloaded=train_dataloaded
    valid_dataloaded=valid_dataloaded
    vec2data=vec2data.to("cuda")

    # Modelleri cihaza taşıyoruz
    num_epochs=40
    train_sets = len(train_dataloaded)
    warmup_epochs=5

    context_opt = torch.optim.AdamW(vec2data.parameters(), lr=1e-4)
    
    scheduler = get_cosine_schedule_with_warmup(context_opt,train_sets* warmup_epochs, train_sets * num_epochs)
    criterion = torch.nn.MSELoss()


    train_losses = []
    model_teacher.train()
    model_student.train()
    
    for epoch in range(num_epochs):
        total_loss = 0.0
        total_loss_val=0.0
        
        for idx, (x, _) in enumerate(tqdm(train_dataloaded, desc=f"Epoch {epoch+1}/{num_epochs}")):
            x = x.to(device)
            
            # context_model'den context, target ve target_indices elde ediliyor
            context= model_teacher(x)
            
            # target_model'den hedef temsiller (target representations) alınıyor
            context_student = model_student(x)
           
            reversed_t=vec2data(context)
            reversed_st=vec2data(context)
            # Her bir hedef temsil için loss hesaplanıyor
            
                # predictor_model ile context temsilleri elde ediliyor.
                # context.detach() ile geriye yayılımın target_model'e gitmesi engelleniyor.
                    

                        # Seçili patch'ler için uygun boyutta gürültü üret
        
                    
                    
                    # Loss: 1 - cosine_similarity, böylece negatif değer oluşması engelleniyor.
            loss = criterion(reversed_t, reversed_st)
                   
            
            
            
            
            
            
           
            # Geriye yayılım
            loss.backward()
            
            # Loss toplamına ekleme (skaler olarak)
            total_loss += loss.item()
            
            # Parametre güncellemesi
            context_opt.step()
            
            
            context_opt.zero_grad()
            
            if device == "cuda":
                torch.cuda.synchronize()
            
            # CUDA kullanıyorsak senkronizasyon
            
        for idx,(images,label) in enumerate(tqdm(valid_dataloaded, desc=f"Epoch {epoch+1}/{num_epochs}")):
            x = images.to(device)
            
            # context_model'den context, target ve target_indices elde ediliyor
            context = model_teacher(x)
            
            # target_model'den hedef temsiller (target representations) alınıyor
            context_student = model_student(x)
            
            with torch.no_grad():
            # Her bir hedef temsil için loss hesaplanıyor
                reversed_t=vec2data(context)
                reversed_st=vec2data(context)
            # Her bir hedef temsil için loss hesaplanıyor
            
                # predictor_model ile context temsilleri elde ediliyor.
                # context.detach() ile geriye yayılımın target_model'e gitmesi engelleniyor.
                    

                        # Seçili patch'ler için uygun boyutta gürültü üret
        
                    
                    
                    # Loss: 1 - cosine_similarity, böylece negatif değer oluşması engelleniyor.
                loss_i = criterion(reversed_t, reversed_st)
                    # predictor_model ile context temsilleri elde ediliyor.
                    # context.detach() ile geriye yayılımın target_model'e gitmesi engelleniyor.
                    

                        # Seçili patch'ler için uygun boyutta gürültü üret
        
                    
                    
                    # Loss: 1 - cosine_similarity, böylece negatif değer oluşması engelleniyor.
                
                   
                    
                    
                
                total_loss_val += loss_i.item()




                
                if device == "cuda":
                    torch.cuda.synchronize()
            # Öğrenme oranı güncellemesi
        scheduler.step()
        
        avg_loss = total_loss / len(train_dataloaded)
        avg_loss_v=total_loss_val/len(valid_dataloaded)
        train_losses.append(avg_loss)
        print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.6f} | Val Loss: {avg_loss_v:.6f}")
    
    print("Finished Training")
    return train_losses


In [22]:
model_student(eleman)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [25]:
eleman

tensor([[[[0.1931, 0.5779, 0.2499,  ..., 0.9439, 0.9774, 0.3176],
          [0.8832, 0.3027, 0.5631,  ..., 0.7996, 0.1233, 0.3603],
          [0.5700, 0.8214, 0.1535,  ..., 0.2904, 0.5488, 0.3477],
          ...,
          [0.0464, 0.6411, 0.6891,  ..., 0.8864, 0.4648, 0.2743],
          [0.4772, 0.7408, 0.1261,  ..., 0.9153, 0.3407, 0.6088],
          [0.2919, 0.8106, 0.1254,  ..., 0.3803, 0.5618, 0.2433]],

         [[0.4280, 0.6545, 0.6576,  ..., 0.0451, 0.5562, 0.2568],
          [0.0669, 0.3496, 0.8679,  ..., 0.1044, 0.4738, 0.6606],
          [0.5818, 0.9670, 0.7874,  ..., 0.3265, 0.0478, 0.6208],
          ...,
          [0.1837, 0.5618, 0.5932,  ..., 0.6050, 0.2745, 0.2886],
          [0.2545, 0.8251, 0.9152,  ..., 0.9774, 0.9630, 0.2012],
          [0.5910, 0.3650, 0.8670,  ..., 0.2470, 0.5347, 0.3237]],

         [[0.8482, 0.8139, 0.6881,  ..., 0.0855, 0.8455, 0.7415],
          [0.4562, 0.8089, 0.2399,  ..., 0.9999, 0.6004, 0.2923],
          [0.8477, 0.3447, 0.1927,  ..., 0

In [27]:
eleman2=torch.stack([i.clone() for i in eleman])

In [2]:
import torch

In [29]:
eleman2.shape

torch.Size([32, 3, 128, 128])

In [3]:
eleman=torch.rand(32,64,64)

In [25]:
diger=eleman.permute(0,2,1).reshape(32,64,8,8)
diger.shape

torch.Size([32, 64, 8, 8])

In [26]:
from torch import nn

In [27]:
conv = nn.Conv2d(
            64, 3, kernel_size=(1,1)
        )

In [21]:
lineer=nn.Linear(8,128)

In [29]:
conv(diger).shape

torch.Size([32, 3, 8, 8])

In [32]:
nn.functional.interpolate(diger,(128,128),mode="bilinear").shape

torch.Size([32, 64, 128, 128])